In [2]:
import os
import torch
import numpy as np
from tqdm import tqdm
from glob import glob
import torch.optim as optim
import random
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import save_image
from torchinfo import summary
from PIL import Image
from Braingen_3d import Model_alphaGAN,Model_VAEGAN,Model_WGAN,Model_alphaWGAN

In [4]:
# 클래스 정의
class_list = ['Normal','Ischemic','Hemorrhagic']

params = {
    # 데이터 설정
    'data_path': '../../data/stroke_CT_preprocessed/',
    'image_count': 10000,
    'image_size': 256,
    'inch': 1,
    'outch': 1,
    'image_count':32,
    # 학습 설정
    'lr': 2e-5,
    'batch_size': 8,
    'epochs': 10000,
    'save_every': 10,
    'save_path': '../../result/edm2/CT',

}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda:0


In [ ]:
# 변환 정의
# trans = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
# ])
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
def transback(x):
    return (x.clamp(-1, 1) + 1) * 0.5

# 이미지 로드
image_paths, image_labels = [], []
for i, cname in enumerate(class_list):
    paths =glob(params['data_path']+class_list[i]+'/**/')
    image_paths.extend(paths)
    image_labels.extend([i] * len(paths))

N = len(image_paths)
C, H, W = params['inch'], params['image_size'], params['image_size']
train_images = torch.zeros((N,params['image_count'], C, H, W), dtype=torch.float32)

print("Loading images into tensor...")
for i, path in enumerate(tqdm(image_paths)):
    img = Image.open(path).convert('L').resize((W, H))
    
    train_images[i] = trans(img)
train_labels = torch.tensor(image_labels, dtype=torch.long)

# 커스텀 Dataset
class CustomDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __getitem__(self, index):
        img = self.images[index]
        lab = self.labels[index]
        # if random.random() > 0.5:
        #     img = transforms.functional.hflip(img)
        # if random.random() > 0.5:
        #     img = transforms.functional.vflip(img)
        return img, lab

    def __len__(self):
        return len(self.images)
def edm_sample_sigma(batch_size, rho, sigma_min, sigma_max, device):
    t = torch.rand(batch_size, device=device)
    sigmas = (sigma_max ** (1 / rho) + t * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))) ** rho
    return sigmas

def get_edm_sigma_schedule(sigma_min, sigma_max, rho, num_steps, device):
    ramp = torch.linspace(0, 1, num_steps, device=device)
    sigmas = (sigma_max ** (1 / rho) + ramp * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))) ** rho
    return sigmas
# DataLoader
train_dataset = CustomDataset(train_images, train_labels)
dataloader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True,drop_last=True)
# 모델 초기화

In [10]:
# 변환 정의
# trans = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
# ])
trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
def transback(x):
    return (x.clamp(-1, 1) + 1) * 0.5

# 이미지 로드
image_paths, image_labels = [], []
for i, cname in enumerate(class_list):
    paths =glob(params['data_path']+class_list[i]+'/**/')
    image_paths.extend(paths)
    image_labels.extend([i] * len(paths))

N = len(image_paths)
C, H, W = params['inch'], params['image_size'], params['image_size']
train_images = torch.zeros((N,params['image_count'], C, H, W), dtype=torch.float32)
for i, path in enumerate(tqdm(image_paths)):
    for j in range(params['image_count']):
        img = Image.open(path).convert('L').resize((W, H))
    
    train_images[i] = trans(img)
train_labels = torch.tensor(image_labels, dtype=torch.long)

  0%|          | 0/3097 [00:00<?, ?it/s]


IsADirectoryError: [Errno 21] Is a directory: '/home/ubuntu/gcubme_ai2/Workspace/YS_Lee/stroke synthesis/data/stroke_CT_preprocessed/Normal/ANO3_1597'

In [9]:
train_images.shape

torch.Size([3097, 32, 1, 256, 256])